In [1]:
from sqlalchemy import create_engine
from collections import Counter
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import os
import random
import time
import pandas as pd

In [2]:
def SeleniumGetPhoto():
    
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-gpu')
    options.add_argument('blink-settings=imagesEnabled=false')
    options.add_argument('--enable-features=OverlayScrollbar')
    
#     chromedriver = '/root/chromedriver'
#     driver = webdriver.Chrome(executable_path=chromedriver, options=options)
    
    chromedriver = './tools/chromedriver.exe'
    driver = webdriver.Chrome(executable_path=chromedriver)

    driver.get("https://smtv.io/admin/login?target=http%3A%2F%2Fsmtv.io%2Fadmin%2FCronJob")
    time.sleep(3)
    driver.find_element_by_xpath("//*[@id='inputEmail']").send_keys('admin')
    driver.find_element_by_xpath("//*[@id='inputPassword']").send_keys('1qaz@WSX')
    
    driver.find_element_by_xpath("/html/body/div/div/div/form/div[4]/button").click()
    time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source,"html.parser")
    driver.close()
    Date = time.strftime("%Y-%m-%d %H:%M", time.localtime())
    return soup, Date

#----------------------------------------------------------------------------

def GetCron(soup,Date):
    table = soup.find("table").find("tbody")
    result = []
    for i in range(len(table.find_all("tr"))):
        CronJob = table.find_all("tr")[i].find_all("td")[0].text.replace(" ","")
        status = table.find_all("tr")[i].find_all("td")[2].text.replace(" ","")
        if status == "SUCCESS":
            status_number = 1
        elif status == "WAITTINGORRUNNING":
            status_number = 2
        else:
            status_number = 0
        if status != "OFF":
            result.append({"CronJob":CronJob,"status":status,"status_number":status_number,"Date":Date})

    return result

#----------------------------------------------------------------------------

#連結資料庫
#範本解釋:create_engine('mysql+mysql_driver://mysql帳號:mysql密碼@機器ip:mysql_port/DB名稱?其他參數', encoding='mysql編碼'
#charset=utf8 資料庫編碼

def List_to_mysql(user,passwd,ip,db_name,table_name):

    engine = create_engine('mysql+mysqlconnector://'+ user +':'+ passwd +'@'+ip+'/'+ db_name +'?charset=utf8', encoding='utf-8')
    con = engine.connect() #建立連結
    
    for item in result:
        df = pd.DataFrame(item, index=[0])
        try:
            df.to_sql(name=table_name,con=con,if_exists='append',index=False) #假設table已存在 就自動往下加入data

        except Exception as e:
            if 'PRIMARY' in str(e):
                pass

    con.close() 
    engine.dispose()

#----------------------------------------------------------------------------

# run code

user = "root"
passwd = "Pn123456"
ip = "192.168.22.110"
db_name = "Solartninc"
table_name = "CronJob"
soup, Date = SeleniumGetPhoto()
result = GetCron(soup,Date)
List_to_mysql(user,passwd,ip,db_name,table_name)
print("time: ",Date,"  insert CronJob done...")

time:  2020-07-25 14:07   insert CronJob done...
